# Projeto 1 - Ciência dos Dados

Nome: Thomas Kassabian

Nome: Maria Carolina Pôrto

Nome: Kaique Pelizer Tinto

Nome: Eduardo Candeias

Atenção: Serão permitidos grupos de quatro pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\othom\OneDrive - Insper - Institudo de Ensino e Pesquisa\insper\Disciplinas\2\CDados\Projetos\CDADOS_Projeto1


Carregando a base de dados com os tweets classificados manualmente:

In [3]:
train = pd.read_csv('dados_treino_QUARTETO_Thomas.csv')
train.head(5)

,Review,NPS
0,Bangkok to Phuket round trip. The lounge at th...,Promoter
1,A real mixed bag with Air New Zealand from Auc...,Passive
2,Second in the queue in business class check-...,Detractor
3,London Heathrow to Riyadh return. Pleasant f...,Promoter
4,Hong Kong to Bangkok. Check-in at the transf...,Promoter


In [4]:
test = pd.read_csv('dados_teste_QUARTETO_Thomas.csv')
test.head(5)

,Review,NPS
0,Melbourne to Singapore on a Singapore Airlines...,Promoter
1,Palermo to Moscow via Rome and I was so unluc...,Detractor
2,Actual flight both ways was good but the handl...,Passive
3,Flew Spirit Airlines from Orlando to Boston....,Detractor
4,"Shockingly poor experience on many levels, as...",Detractor


### Limpando a base de dados

In [5]:
characs = ',.!?:;/()'
#stopwords = ['t','they', 'a', 'herself', 'beverages', 'claim', 'window', 'miles', 'she', 'stopover', 'policies', 'his', 'have', 'who', 'be', 'reward', 'bags', 'claims', 'to', 'economy', 'in', 'or', 'program', 'before', 'are', 'airlines', 'baggage', 'ratings', 'check-in', 'crew', 'that', 'connecting', 'status', 'with', 'plane', 'non-stop', 'him', 'because', 'same', 'than', 'review', 'some', 'more', 'airport', 'arrival', 'airline', 'first', 'does', 'surcharge', 'both', 'tax', 'if', 'on', 'experience', 'so', 'most', 'security', 'no', 'middle', 'above', 'business', 'a', 'up', 'complaint', 'of', 'under', 'gate', 'prices', 'meal', 'cabin', 'delay', 'me', 'himself', 'myself', 'your', 'yours', 'my', 'them', 'duty-free', 'were', 'is', 'staff', 'turbulence', 'then', 'each', 'other', 'such', 'membership', 'pre-flight', 'can', 'did', 'comfort', 'cost', 'taxes', 'which', 'meals', 'hers', 'jet', 'as', 'off', 'departure', 'attendant', 'aisle', 'am', 'been', 'legroom', 'why', 'their', 'her', 'ground', 'feedback', 'from', 'surcharges', 'too', 'doing', 'until', 'drinks', 'alliance', 'during', 'itself', 'now', 'promotions', 'discounts', 'not', 'there', 'being', 'between', 'about', 'very', 'offer', 'ours', 'further', 'attendants', 'weather', 'in-flight', 'connect', 'price', 'below', 'route', 'snack', 'service', 'class', 'trip', 'frequent', 'at', 'themselves', 'against', 'schedule', 'post-flight', 'our', 'flyer', 'spokes', 'board', 'entertainment', 'code-share', 'aviation', 'yourself', 'these', 'you', 'charges', 'do', 'upgrade', 'aircraft', 'mile', 'any', 'tickets', 'amenity', 'member', 'just', 'book', 'should', 'has', 'when', 'complaints', 'out', 'policy', 'charge', 'we', 'return', 'layover', 'beverage', 'whom', 'while', 'cancel', 'how', 'don', 'destination', 'travel', 'once', 'here', 'maintenance', 'luggage', 'food', 'nor', 'bins', 'offers', 'terminal', 'discount', 'deal', 'over', 'flight', 'will', 'onboard', 'connection', 'an', 'few', 'sale', 'all', 'procedure', 'lounge', 'passengers', 'fees', 'traveller', 'theirs', 's', 'hub', 'fare', 'deals', 'down', 'ourselves', 'they', 'into', 'reviews', 'customer', 'passenger', 'pilot', 'it', 'overhead', 'he', 'its', 'and', 'had', 'insurance', 'for', 'elite', 'booking', 'snacks', 'i', 'partnership', 'checked', 'carry-on', 'reservation', 'journey', 'only', 'seat', 'seats', 'ticket', 'pilots', 'this', 'having', 'bag', 'bin', 'amenities', 'delayed', 'was', 'promotion', 'fee', 'routes', 'through', 'again', 'the', 'direct', 'after', 'cancelled', 'points', 'where', 'safety', 'procedures', 'drink', 'travellers', 'by', 'sales', 'own', 'but', 'what', 'yourselves', 'boarding', 'those', 'rating']

for c in characs:
    train['Review'] = train['Review'].str.replace(c, '').str.lower()
    test['Review'] = test['Review'].str.replace(c, '').str.lower()

#for w in stopwords:
    #train['Review'] = train['Review'].str.replace(' ' + w + ' ', ' ')
    #test['Review'] = test['Review'].str.replace(' ' + w + ' ', ' ')

### Separando os dados em series

In [6]:
# Separando as palavras por classificação
classificacoes = ['Promoter', 'Detractor', 'Passive']
palavras_dict = {}

for e in classificacoes:
    palavras_lista = []

    for linha in train[train['NPS'] == e]['Review']:
        palavras = linha.split()
        for palavra in palavras:
            palavras_lista.append(palavra)
    

    palavras_dict[e] = palavras_lista

palavras_promoter = pd.Series(palavras_dict['Promoter'])
palavras_detractor = pd.Series(palavras_dict['Detractor'])
palavras_passive = pd.Series(palavras_dict['Passive'])
palavras_train = pd.Series(train['Review'].str.split().sum())


### Contagem

In [7]:
# Frequência absoluta das categorias
tabela_categorias = train['NPS'].value_counts()

# Frequência relativa das categorias
tabela_categorias_relativa = train['NPS'].value_counts(True)

# Frequência absoluta das palavras
tabela_train = palavras_train.value_counts()
tabela_promoter = palavras_promoter.value_counts()
tabela_detractor = palavras_detractor.value_counts()
tabela_passive = palavras_passive.value_counts()

# Frequência relativa das palavras
tabela_train_relativa = palavras_train.value_counts(True)
tabela_promoter_relativa = palavras_promoter.value_counts(True)
tabela_detractor_relativa = palavras_detractor.value_counts(True)
tabela_passive_relativa = palavras_passive.value_counts(True)

tabela_categorias_relativa

NPS
Detractor    0.467857
Promoter     0.368810
Passive      0.163333
Name: proportion, dtype: float64

___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [8]:
def calc_prob_palavra(palavra, tab_NPS, len_serie_NPS):
    prob = 1
    if palavra in tab_NPS:
        prob *= ((tab_NPS[palavra] + 1)/(len_serie_NPS+ len(tabela_train)))
    else:
        prob *= (1/(len_serie_NPS + len(tabela_train)))

    return prob

In [9]:
for frase in train['Review']:
    prob_detractor = 1
    prob_promoter = 1
    prob_passive = 1
    for palavra in frase.split():
        prob_promoter *= calc_prob_palavra(palavra, tabela_promoter, len(palavras_promoter))
        prob_detractor *= calc_prob_palavra(palavra, tabela_detractor, len(palavras_detractor))
        prob_passive *= calc_prob_palavra(palavra, tabela_passive, len(palavras_passive))

    P_frase_dado_D = prob_detractor*tabela_categorias_relativa['Detractor']
    P_frase_dado_P = prob_promoter*tabela_categorias_relativa['Promoter']
    P_frase_dado_Pa = prob_passive*tabela_categorias_relativa['Passive']

    if P_frase_dado_D > P_frase_dado_P and P_frase_dado_D > P_frase_dado_Pa:
        train['Boot'] = 'Detractor'
    elif P_frase_dado_P > P_frase_dado_D and P_frase_dado_P > P_frase_dado_Pa:
        train['Boot'] = 'Promoter'
    else:
        train['Boot'] = 'Passive'
    

train.head(15)

,Review,NPS,Boot
0,bangkok to phuket round trip the lounge at the...,Promoter,Promoter
1,a real mixed bag with air new zealand from auc...,Passive,Promoter
2,second in the queue in business class check-...,Detractor,Promoter
3,london heathrow to riyadh return pleasant fl...,Promoter,Promoter
4,hong kong to bangkok check-in at the transfe...,Promoter,Promoter
5,hkg-svo-hkg a330 economy both legs slightly de...,Passive,Promoter
6,this flight from bodhgaya to delhi was the l...,Passive,Promoter
7,they kept us on an airplane for 2 hours initia...,Detractor,Promoter
8,kuala lumpur to krabi despite that airasia hav...,Passive,Promoter
9,frankfurt to lima via madrid with air europa f...,Passive,Promoter


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**